In [ ]:
import random
import time
from openpyxl import Workbook

# Define the recipes as a list from 1 to 10
recipes = list(range(1, 11))

# Define the recipe eligibility for each factory
factory_eligibility = {
    'F1': random.sample(recipes, int(0.5 * len(recipes))),  # 50% of total recipes are eligible for F1
    'F2': random.sample(recipes, int(0.7 * len(recipes))),  # 70% of total recipes are eligible for F2
    'F3': recipes  # 100% recipes are eligible for F3
}

# Generate random orders with 1-4 recipes
def generate_orders(num_orders):
    orders = []
    for n in range(num_orders):
        recipe_ids = random.sample(recipes, random.randint(1, 4))
        orders.append({'id': len(orders) + 1, 'recipe_ids': recipe_ids})
    return orders

# Get the eligible factories for each order based on the recipes
def get_eligible_factories(order):
    eligible_factories = []
    for factory, eligible_recipes in factory_eligibility.items():
        if all(recipe_id in eligible_recipes for recipe_id in order['recipe_ids']):
            eligible_factories.append(factory)
    return eligible_factories

# Generate sample orders for LD5 (F) and LD3 (A)
num_orders = 20
orders_ld5 = generate_orders(num_orders)
orders_ld3 = generate_orders(num_orders)

# Add eligible factories to each order
for order in orders_ld5 + orders_ld3:
    order['eligible_factories'] = get_eligible_factories(order)

# Define factory capacities
factory_capacities = {
    'F1': int(num_orders / 3),  # Serve a third of total orders
    'F2': int(num_orders * 2 / 3),  # Serve two thirds of total orders
    'F3': float('inf')  # Catch-all factory with infinite capacity
}

# Initial allocation (orders with fewer eligible factories are allocated first, as they have limited options)
def initial_allocation(orders, factory_capacities):
    allocation = {factory: [] for factory in factory_capacities}
    sorted_orders = sorted(orders, key=lambda x: len(x['eligible_factories']))
    for order in sorted_orders:
        allocated = False
        for factory in order['eligible_factories']:
            if len(allocation[factory]) < factory_capacities[factory]:
                if all(recipe_id in factory_eligibility[factory] for recipe_id in order['recipe_ids']):
                    allocation[factory].append(order)
                    allocated = True
                    break
        if not allocated:
            allocation['F3'].append(order)
    return allocation

# Calculate WMAPE site
def calculate_wmape_site(allocation_ld5, allocation_ld3):
    total_abs_diff = 0
    total_ld3_items = 0

    for factory in ['F1', 'F2']:
        recipe_counts_ld5 = {}
        recipe_counts_ld3 = {}

        for order in allocation_ld5[factory]:
            for recipe_id in order['recipe_ids']:
                recipe_counts_ld5[recipe_id] = recipe_counts_ld5.get(recipe_id, 0) + 1

        for order in allocation_ld3[factory]:
            for recipe_id in order['recipe_ids']:
                recipe_counts_ld3[recipe_id] = recipe_counts_ld3.get(recipe_id, 0) + 1
                total_ld3_items += 1

        for recipe_id in set(recipe_counts_ld5.keys()) | set(recipe_counts_ld3.keys()):
            ld5_count = recipe_counts_ld5.get(recipe_id, 0)
            ld3_count = recipe_counts_ld3.get(recipe_id, 0)
            total_abs_diff += abs(ld5_count - ld3_count)

    if total_ld3_items == 0:
        return float('inf')  # Return a large positive number as default
    else:
        wmape_site = total_abs_diff / total_ld3_items
        return wmape_site

# Calculate WMAPE global
def calculate_wmape_global(allocation_ld5, allocation_ld3):
    total_abs_diff = 0
    total_ld3_items = 0

    recipe_counts_ld5 = {}
    recipe_counts_ld3 = {}

    for factory in allocation_ld5:
        for order in allocation_ld5[factory]:
            for recipe_id in order['recipe_ids']:
                recipe_counts_ld5[recipe_id] = recipe_counts_ld5.get(recipe_id, 0) + 1

    for factory in allocation_ld3:
        for order in allocation_ld3[factory]:
            for recipe_id in order['recipe_ids']:
                recipe_counts_ld3[recipe_id] = recipe_counts_ld3.get(recipe_id, 0) + 1
                total_ld3_items += 1

    for recipe_id in set(recipe_counts_ld5.keys()) | set(recipe_counts_ld3.keys()):
        ld5_count = recipe_counts_ld5.get(recipe_id, 0)
        ld3_count = recipe_counts_ld3.get(recipe_id, 0)
        total_abs_diff += abs(ld5_count - ld3_count)

    wmape_global = total_abs_diff / total_ld3_items
    return wmape_global

# Apply Genetic Algorithm
def apply_genetic_algorithm(allocation_ld3, orders_ld3, factory_capacities, population_size, num_generations, mutation_rate):
    def create_individual():
        individual = {factory: [] for factory in factory_capacities}
        for order in orders_ld3:
            eligible_factories = [f for f in order['eligible_factories'] if len(individual[f]) < factory_capacities[f]]
            if eligible_factories:
                factory = random.choice(eligible_factories)
                individual[factory].append(order)
            else:
                individual['F3'].append(order)
        return individual

    def evaluate_fitness(individual):
        return -calculate_wmape_site(allocated_orders_ld5, individual)

    def crossover(parent1, parent2):
        child = {factory: [] for factory in factory_capacities}
        for factory in ['F1', 'F2']:
            p1_orders = parent1[factory]
            p2_orders = parent2[factory]
            size = min(len(p1_orders), len(p2_orders))
            for i in range(size):
                if random.random() < 0.5:
                    child[factory].append(p1_orders[i])
                else:
                    child[factory].append(p2_orders[i])
    
        # Randomly select orders from either parent for 'F3' factory
        p1_orders_f3 = parent1['F3']
        p2_orders_f3 = parent2['F3']
        combined_orders_f3 = p1_orders_f3 + p2_orders_f3
        child['F3'] = random.sample(combined_orders_f3, len(combined_orders_f3))
    
        return child

    def mutate(individual):
        for factory in ['F1', 'F2']:
            orders = individual[factory]
            for i in range(len(orders)):
                if random.random() < mutation_rate:
                    eligible_factories = [f for f in orders[i]['eligible_factories'] if f != factory]
                    if eligible_factories:
                        new_factory = random.choice(eligible_factories)
                        if len(individual[new_factory]) < factory_capacities[new_factory]:
                            individual[new_factory].append(orders.pop(i))
                            break
        return individual

    population = [create_individual() for _ in range(population_size)]
    best_individual = max(population, key=evaluate_fitness)
    best_fitness = evaluate_fitness(best_individual)

    for generation in range(num_generations):
        offspring = []
        for r in range(population_size // 2):
            parent1, parent2 = random.sample(population, 2)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])

        offspring = [mutate(individual) for individual in offspring]
        population = offspring

        best_individual_in_generation = max(population, key=evaluate_fitness)
        best_fitness_in_generation = evaluate_fitness(best_individual_in_generation)
        if best_fitness_in_generation > best_fitness:
            best_individual = best_individual_in_generation
            best_fitness = best_fitness_in_generation

    return best_individual

# Initial allocation
allocated_orders_ld5 = initial_allocation(orders_ld5, factory_capacities)
allocated_orders_ld3 = initial_allocation(orders_ld3, factory_capacities)

# Calculate initial WMAPE site and WMAPE global
initial_wmape_site = calculate_wmape_site(allocated_orders_ld5, allocated_orders_ld3)
initial_wmape_global = calculate_wmape_global(allocated_orders_ld5, allocated_orders_ld3)
print(f"Initial WMAPE site: {initial_wmape_site:.2f}")
print(f"WMAPE global: {initial_wmape_global:.2f}")

# Apply Genetic Algorithm
population_size = 50
num_generations = 100
mutation_rate = 0.1

start_time = time.time()  # Start measuring execution time

optimized_allocation_ld3 = apply_genetic_algorithm(allocated_orders_ld3, orders_ld3, factory_capacities, population_size, num_generations, mutation_rate)

end_time = time.time()  # End measuring execution time
execution_time = end_time - start_time

# Calculate optimized WMAPE site
optimized_wmape_site = calculate_wmape_site(allocated_orders_ld5, optimized_allocation_ld3)
print(f"Optimized WMAPE site: {optimized_wmape_site:.2f}")
print(f"Execution time: {execution_time:.2f} seconds")

# Export order data and allocation solutions to Excel
def export_to_excel(orders_ld5, orders_ld3, allocated_orders_ld5, allocated_orders_ld3, optimized_allocation_ld3):
    workbook = Workbook()
    sheet_ld5_orders = workbook.active
    sheet_ld5_orders.title = 'LD5 Orders'

    sheet_ld3_orders = workbook.create_sheet(title='LD3 Orders')
    sheet_allocation_ld5 = workbook.create_sheet(title='LD5 Allocation')
    sheet_allocation_ld3 = workbook.create_sheet(title='LD3 Allocation')
    sheet_optimized_allocation_ld3 = workbook.create_sheet(title='Optimized LD3 Allocation')

    order_headers = ['Order ID', 'Recipe IDs', 'Eligible Factories']
    sheet_ld5_orders.append(order_headers)
    sheet_ld3_orders.append(order_headers)

    allocation_headers = ['Factory', 'Allocated Orders', 'Recipe IDs']
    sheet_allocation_ld5.append(allocation_headers)
    sheet_allocation_ld3.append(allocation_headers)
    sheet_optimized_allocation_ld3.append(allocation_headers)

    # Write order data for LD5
    for order in orders_ld5:
        row = [order['id'], ', '.join(map(str, order['recipe_ids'])), ', '.join(order['eligible_factories'])]
        sheet_ld5_orders.append(row)

    # Write order data for LD3
    for order in orders_ld3:
        row = [order['id'], ', '.join(map(str, order['recipe_ids'])), ', '.join(order['eligible_factories'])]
        sheet_ld3_orders.append(row)

    # Write allocation data for LD5
    for factory, orders in allocated_orders_ld5.items():
        for order in orders:
            row = [factory, order['id'], ', '.join(map(str, order['recipe_ids']))]
            sheet_allocation_ld5.append(row)

    # Write allocation data for LD3
    for factory, orders in allocated_orders_ld3.items():
        for order in orders:
            row = [factory, order['id'], ', '.join(map(str, order['recipe_ids']))]
            sheet_allocation_ld3.append(row)

    # Write optimized allocation data for LD3
    for factory, orders in optimized_allocation_ld3.items():
        for order in orders:
            row = [factory, order['id'], ', '.join(map(str, order['recipe_ids']))]
            sheet_optimized_allocation_ld3.append(row)

    # Save the workbook
    workbook.save('GA.xlsx')

# Export the order data and allocation solutions to Excel
export_to_excel(orders_ld5, orders_ld3, allocated_orders_ld5, allocated_orders_ld3, optimized_allocation_ld3)

Initial WMAPE site: 1.00
WMAPE global: 0.47
